In [1]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn import preprocessing
import plotnine as p9


In [2]:
X_train = pd.read_csv("./data/external/train_logs.csv")

X_train = (
    X_train
    .sort_values(["id", "event_id"], ascending=[True, True])
    )

In [3]:
PUNCTUATION = X_train.loc[(
    (X_train['activity'] == 'Input')
    & (~ X_train['text_change'].isin(['q', ' ']))
    ), 'text_change'].unique()

In [4]:
# 'Move From' activity recorded with low-level cursor loc details
# extract bigger-picture 'Move From'
# QUESTION: what's the difference between Move From, and a cut+paste?
X_train['activity_detailed'] = X_train['activity']
X_train.loc[X_train['activity'].str.contains('Move From'), 'activity'] = 'Move'

In [5]:
# no explicit record for a pause. pauses are omitted.
PAUSE_THRESHOLD_MS = 1000

X_train['up_time_lag1'] = (
    X_train
    .groupby(['id'])
    ['up_time']
    .shift(1)
    )
# latency does not mean a meaningful pause
X_train['latency_time'] = (
    X_train['down_time'] - X_train['up_time_lag1']
    )

X_train['preceding_pause_time'] = X_train['latency_time']
# first record lacks preceding_pause_time: that's time before first key press
X_train.loc[X_train['event_id'] == 1, 'preceding_pause_time'] = X_train['down_time']
# expect some negative pause times -- interpret as, no real pause
has_no_real_pause = X_train['preceding_pause_time'] <= PAUSE_THRESHOLD_MS
X_train.loc[has_no_real_pause, 'preceding_pause_time'] = None
# not obvious how to tag "initial planning pause" 
X_train['preceding_pause_time_start_window'] = X_train['preceding_pause_time']
X_train.loc[X_train['up_time'] > 5 * 60 * 1000, 'preceding_pause_time_start_window'] = None

X_train['total_pause_time'] = (
    X_train
    .groupby(['id'])
    ['preceding_pause_time']
    .transform('sum')
    )
X_train['rolling_pause_time'] = (
    X_train
    .groupby(['id'])
    ['preceding_pause_time']
    .cumsum()
    )
X_train['rolling_pause_time_fraction'] = (
    X_train['rolling_pause_time'] / X_train['total_pause_time']
    )

# summarize pause distr
MS_IN_PAUSE_BUCKET_MAX = 200e3
PAUSE_BUCKET_STEP_MS = 500

X_train['preceding_pause_time_bucket'] = pd.cut(
    X_train['preceding_pause_time'],
    bins=np.arange(
        0, 
        MS_IN_PAUSE_BUCKET_MAX,
        PAUSE_BUCKET_STEP_MS
        )
    )

# X_train['preceding_pause_time_bucket'].value_counts()

# WARNING: this representation of pause distribution is dense & large
# a few parameters from distribution model far more succinct

In [6]:
# if pause exceeds threshold duration, a "burst" has ended
MS_PER_S = 1000
SECONDS_PER_BURST = 2

X_train['is_new_burst_start'] = (
    X_train['preceding_pause_time'] > MS_PER_S * SECONDS_PER_BURST
    ).astype(int)
X_train['is_new_burst_start'][0] = 1
X_train['burst_id'] = (
    X_train
    .groupby(['id'])
    ['is_new_burst_start']
    .cumsum()
    )
X_train['burst_time_start'] = (
    X_train
    .groupby(['id', 'burst_id'])
    ['down_time']
    .transform('min')
    )
X_train['burst_time_end'] = (
    X_train
    .groupby(['id', 'burst_id'])
    ['up_time']
    .transform('max')
    )
X_train['burst_time_duration'] = (
    X_train['burst_time_end'] - X_train['burst_time_start']
    )

/tmp/ipykernel_42653/101131515.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [7]:
# one-way cursor movement might be most productive
# jumping around is choppy
X_train['cursor_position_lag1'] = (
    X_train
    .groupby(['id'])
    ['cursor_position']
    .shift(1)
    )

X_train['has_cursor_position_moved_right'] = (
    X_train['cursor_position'] > X_train['cursor_position_lag1']
    ).astype(int)

# farthest position cursor has _edited_, with recorded input
X_train['cursor_position_cummax'] = (
    X_train
    .groupby(['id'])
    ['cursor_position']
    .cummax()
    )
X_train.loc[X_train['activity'] != 'Input', 'cursor_position_cummax'] = None
X_train['cursor_position_cummax'] = (
    X_train
    .groupby(['id'])
    ['cursor_position_cummax']
    .ffill()
    )

X_train['cursor_position_vs_max'] = (
    X_train['cursor_position'] - X_train['cursor_position_cummax']
    )

In [8]:
# word count offers a productivity measure
X_train['word_count_lag1'] = (
    X_train
    .groupby(['id'])
    ['word_count']
    .shift(1)
    )

X_train['word_count_delta_event'] = (
    X_train['word_count'] - X_train['word_count_lag1']
    )

X_train['word_count_delta_burst'] = (
    X_train
    .groupby(['id', 'burst_id'])
    ['word_count_delta_event']
    .transform('sum')
    )
# de-duplicate to one value per burst -- easier for downstream aggregation
X_train['word_count_delta_burst_thin'] = X_train['word_count_delta_burst']
X_train.loc[X_train['is_new_burst_start'] == 0, 'word_count_delta_burst_thin'] = None


# word length offers a content quality measure.
# hard to track entire words sequence in rolling fashion.
    # every word's length, in a list of one element per word?  
# more tractable to track very latest string

is_edit_to_latest_string = X_train['cursor_position_vs_max'] == 0

X_train['is_new_latest_string_start'] = (
    is_edit_to_latest_string
    & (X_train['activity'] == "Input")
    & (X_train["text_change"] == ' ')
    )

X_train['is_latest_string_end'] = (
    X_train
    .groupby(['id'])
    ['is_new_latest_string_start']
    .shift(-1)
    # last process records
    .fillna(True)
    )

X_train['n_alphanum_char_added_to_latest_string'] = 0
is_alphanumeric_addition = (
    (X_train['activity'] == "Input")
    & (X_train["text_change"] == 'q')
    )
X_train.loc[
    (is_alphanumeric_addition & is_edit_to_latest_string), 
    'n_alphanum_char_added_to_latest_string'
    ] = 1
is_alphanumeric_subtraction = (
    (X_train['activity'] == "Remove/Cut")
    & (X_train['up_event'] == 'Backspace')
    & (X_train["text_change"] == 'q')
    )
X_train.loc[
    (is_alphanumeric_subtraction & is_edit_to_latest_string), 
    'n_alphanum_char_added_to_latest_string'
    ] = -1
# example: 2nd string, 2 characters in.
# considering cumsum for each character in 2nd string, 
# subtract those characters from 1st
X_train['rolling_length_latest_string'] = (
    X_train
    .groupby(['id'])
    ['n_alphanum_char_added_to_latest_string']
    .cumsum() 
    ) - (
    X_train
    .groupby(['id'])
    ['n_alphanum_char_added_to_latest_string']
    .cumsum()
    .where(X_train['is_new_latest_string_start']) 
    .ffill()
    .fillna(0)
    )

X_train['length_latest_string'] = None
X_train.loc[
    X_train['is_latest_string_end'], 
    'length_latest_string'
    ] = X_train['rolling_length_latest_string']

In [9]:
# if thoughts aren't separated by punctuation, writing won't score well
X_train['is_thought_delimiting_punctuation'] = (
    (X_train['text_change'] == ".")
    | (X_train['text_change'] == ". ")
    | (X_train['text_change'] == ",")
    | (X_train['text_change'] == "-")
    | (X_train['text_change'] == "!")
    | (X_train['text_change'] == ";")
    | (X_train['text_change'] == "?")
    | (X_train['text_change'] == ":")
    ).astype(int)

X_train['is_special_punctuation'] = (
    (X_train['text_change'] == "=")
    | (X_train['text_change'] == "/")
    | (X_train['text_change'] == "\\")
    | (X_train['text_change'] == "(")
    | (X_train['text_change'] == ")")
    | (X_train['text_change'] == "\n")
    | (X_train['text_change'] == "[")
    | (X_train['text_change'] == "]")
    | (X_train['text_change'] == ">")
    | (X_train['text_change'] == "<")
    | (X_train['text_change'] == "$")
    | (X_train['text_change'] == "*")
    | (X_train['text_change'] == "&")
)

In [10]:
# windows allow for time-sequence features
TOTAL_MIN = 30
SECONDS_PER_MIN = 60
SECONDS_PER_WINDOW = 30

X_train['window_30s'] = pd.cut(
    X_train['down_time'],
    bins=np.arange(
        0, 
        TOTAL_MIN * SECONDS_PER_MIN * MS_PER_S + 5*MS_PER_S*2, 
        SECONDS_PER_WINDOW * MS_PER_S
        )
    )

In [11]:
ACTIVITY_CATEGORIES = ['Nonproduction', 'Input', 'Remove/Cut', 'Replace', 'Paste', 'Move']

pipeline_activity_onehot = ColumnTransformer(
    transformers=[(
        'onehot_encode', 
        preprocessing.OneHotEncoder(
            categories=[ACTIVITY_CATEGORIES], 
            sparse=False, 
            handle_unknown='infrequent_if_exist'
            ),
        ["activity"]
    )],
    remainder='passthrough',
    verbose_feature_names_out=False
    )
pipeline_activity_onehot.fit(X_train)
original_categorical = X_train['activity']

X_train_dtypes = X_train.dtypes.to_dict()
X_train = pipeline_activity_onehot.transform(X_train)
X_train = pd.DataFrame(X_train, columns=pipeline_activity_onehot.get_feature_names_out())
X_train = pd.concat([X_train, original_categorical], axis=1)
X_train = X_train.astype(X_train_dtypes)

/home/alex/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.


In [12]:
for activity in ACTIVITY_CATEGORIES:

    X_train['burst_events_' + activity] = (
        X_train
        .groupby(['id', 'burst_id'])
        ['activity_' + activity]
        .transform('sum')
        ).astype(float)
    
X_train['burst_type'] = (
    X_train
    [['burst_events_' + activity for activity in ACTIVITY_CATEGORIES]]
    .idxmax(axis=1)
    )
X_train['burst_type'] = (
    X_train['burst_type']
    .str
    .replace("burst_events_", "", regex=True)
    )

In [13]:
pipeline_burst_type_onehot = ColumnTransformer(
    transformers=[(
        'onehot_encode', 
        preprocessing.OneHotEncoder(
            categories=[ACTIVITY_CATEGORIES], 
            sparse=False, 
            handle_unknown='infrequent_if_exist'
            ),
        ["burst_type"]
    )],
    remainder='passthrough',
    verbose_feature_names_out=False
    )
pipeline_burst_type_onehot.fit(X_train)
original_categorical = X_train['burst_type']

X_train_dtypes = X_train.dtypes.to_dict()
X_train = pipeline_burst_type_onehot.transform(X_train)
X_train = pd.DataFrame(X_train, columns=pipeline_burst_type_onehot.get_feature_names_out())
X_train = pd.concat([X_train, original_categorical], axis=1)
X_train = X_train.astype(X_train_dtypes)

for activity in ACTIVITY_CATEGORIES:

    X_train['is_new_burst_start_' + activity] = (
        X_train['is_new_burst_start'] * 
        X_train['burst_type_' + activity]
        )

/home/alex/anaconda3/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.


In [14]:
X_train = X_train[[
    "id",
    "event_id",
    "window_30s",
    "burst_id",
    "burst_type",
    "burst_type_Nonproduction",
    "burst_type_Input",
    "burst_type_Remove/Cut",
    "burst_type_Replace",
    "burst_type_Paste",
    "burst_type_Move",
    "is_new_burst_start",
    "is_new_burst_start_Nonproduction",
    "is_new_burst_start_Input",
    "is_new_burst_start_Remove/Cut",
    "is_new_burst_start_Replace",
    "is_new_burst_start_Paste",
    "is_new_burst_start_Move",
    "burst_time_start",
    "burst_time_end",
    "burst_time_duration",
    "burst_events_Nonproduction",
    "burst_events_Input",
    "burst_events_Remove/Cut",
    "burst_events_Replace",
    "burst_events_Paste",
    "burst_events_Move",
    "word_count_delta_burst",
    "word_count_delta_burst_thin",

    "down_time",
    "up_time",	
    "action_time",	
    "activity_detailed",
    "activity",	
    "activity_Nonproduction",
    "activity_Input",
    "activity_Remove/Cut",
    "activity_Replace",
    "activity_Paste",
    "activity_Move",
    "down_event",	
    "up_event",	
    "text_change",
    "is_thought_delimiting_punctuation",
    "cursor_position",	
    "word_count",

    "cursor_position_vs_max",
    "cursor_position_cummax",
    "has_cursor_position_moved_right",

    "is_new_latest_string_start",
    "is_latest_string_end",
    "n_alphanum_char_added_to_latest_string",
    "rolling_length_latest_string",
    "length_latest_string",

    "word_count_lag1",
    "word_count_delta_event",

    "up_time_lag1",
    "latency_time",
    "preceding_pause_time",
    "preceding_pause_time_start_window",
    "preceding_pause_time_bucket",
    "rolling_pause_time",
    "rolling_pause_time_fraction",
    "total_pause_time"
    ]]


In [15]:
X_train.to_pickle("./data/processed/train_logs_enriched.pkl")

In [16]:
(
    X_train
    .query("id == '001519c8'")
    .to_csv("./data/X_train_enriched_case.csv", index=False)
)

In [149]:
event_vars_sum = (
    ['activity_' + x for x in ACTIVITY_CATEGORIES] 
    + ['is_new_burst_start'] 
    + ['is_new_burst_start_' + x for x in ACTIVITY_CATEGORIES]
    + ["is_thought_delimiting_punctuation"]
    )

events_sum_over_time = (
    X_train
    .groupby('id')
    [event_vars_sum]
    .agg(sum)
    )

events_sum_over_time['delete_insert_ratio'] = (
    events_sum_over_time['activity_Remove/Cut'] / 
    events_sum_over_time['activity_Input'] 
    )

conti_vars_sum = (
    ['word_count_delta_event']
    + ["preceding_pause_time"]
    )

conti_sum_over_time = (
    X_train
    .groupby('id')
    [conti_vars_sum]
    .agg(sum)
    )

sums_over_time = pd.merge(
    events_sum_over_time,
    conti_sum_over_time,
    how='left',
    left_index=True,
    right_index=True
)

In [150]:
centrals_over_time = (
    X_train
    .groupby('id')
    .agg(
        latency_time_p50 = ('latency_time', np.median),
        pause_time_p50 = ('preceding_pause_time', np.median),
        has_cursor_position_moved_right_mean = ('has_cursor_position_moved_right', 'mean'),
        word_count_delta_burst_mean = ('word_count_delta_burst_thin', 'mean'),
        word_count_delta_burst_p50 = ('word_count_delta_burst_thin', np.median),
        cursor_position_vs_max_avg = ('cursor_position_vs_max', 'mean'),
        length_latest_string_mean = ('length_latest_string', 'mean'),
        length_latest_string_stddev = ('length_latest_string', np.std)
        )
    )

In [151]:
extremes_over_time = (
    X_train
    .groupby('id')
    .agg(
        pause_time_max=('preceding_pause_time', 'max'),
        initial_pause_time_max=('preceding_pause_time_start_window', 'max'),
        # approximation to, next longest pause after first long planning pause
        pause_time_p99=('preceding_pause_time', lambda x: x.quantile(0.99)),
        word_count_delta_burst_max=('word_count_delta_burst_thin', 'max'),
        total_time=('up_time', 'max'),
        length_latest_string_max=('length_latest_string', 'max'),
        latency_time_min=('latency_time', 'min')
        )
    )

extremes_over_time['is_initial_pause_max_pause'] = (
    extremes_over_time['pause_time_max'] == 
    extremes_over_time['initial_pause_time_max']
    ).astype(int)

In [152]:
from scipy.stats import lognorm

pause_distr_summary_subjects = []

for X_train_subject in [x for _, x in X_train.groupby('id')]:

    shape, location, scale = lognorm.fit(X_train_subject['preceding_pause_time'].dropna())

    pause_distr_summary = pd.DataFrame({
        'pauses_lognorm_shape': [shape], 
        'pauses_lognorm_location': [location],
        'pauses_lognorm_scale': [scale]
        })
    pause_distr_summary.index = [X_train_subject['id'].iloc[0]]
    
    pause_distr_summary_subjects.append(pause_distr_summary)

distr_params_over_time = pd.concat(pause_distr_summary_subjects, axis=0)

In [153]:
aggregates_over_time = pd.merge(
    sums_over_time, 
    centrals_over_time,
    how='left',
    left_index=True,
    right_index=True
    )

aggregates_over_time = pd.merge(
    aggregates_over_time, 
    extremes_over_time,
    how='left',
    left_index=True,
    right_index=True
    )

aggregates_over_time = pd.merge(
    aggregates_over_time, 
    distr_params_over_time,
    how='left',
    left_index=True,
    right_index=True
    )

In [154]:
for var in event_vars_sum:

    aggregates_over_time[var + '_per_s'] = (
        1000 * (aggregates_over_time[var] / aggregates_over_time['total_time'])
        )

aggregates_over_time = (
    aggregates_over_time
    .assign(
        keystroke_speed = lambda x: (x.activity_Input + x['activity_Remove/Cut']) / x.total_time,
        words_per_thought_delimiting_punctuation = lambda x: x.word_count_delta_event / x.is_thought_delimiting_punctuation,
        )
    )

In [155]:
aggregates_over_time.head()

,activity_Nonproduction,activity_Input,activity_Remove/Cut,activity_Replace,activity_Paste,activity_Move,is_new_burst_start,is_new_burst_start_Nonproduction,is_new_burst_start_Input,is_new_burst_start_Remove/Cut,...,is_new_burst_start_per_s,is_new_burst_start_Nonproduction_per_s,is_new_burst_start_Input_per_s,is_new_burst_start_Remove/Cut_per_s,is_new_burst_start_Replace_per_s,is_new_burst_start_Paste_per_s,is_new_burst_start_Move_per_s,is_thought_delimiting_punctuation_per_s,keystroke_speed,words_per_thought_delimiting_punctuation
id,,,,,,,,,,,,,,,,,,,,,
001519c8,120.0,2010.0,417.0,7.0,0.0,3.0,125,23.0,89.0,13.0,...,0.069369,0.012764,0.04939,0.007214,0.0,0.0,0.0,0.023863,0.001347,5.930233
0022f953,254.0,1938.0,260.0,1.0,1.0,0.0,81,32.0,41.0,8.0,...,0.045277,0.017887,0.022918,0.004472,0.0,0.0,0.0,0.028508,0.001229,6.274510
0042269b,175.0,3515.0,439.0,7.0,0.0,0.0,78,30.0,46.0,2.0,...,0.044026,0.016933,0.025964,0.001129,0.0,0.0,0.0,0.028222,0.002232,8.080000
0059420b,99.0,1304.0,151.0,1.0,1.0,0.0,88,9.0,77.0,2.0,...,0.062657,0.006408,0.054825,0.001424,0.0,0.0,0.0,0.011392,0.001036,12.875000
0075873a,72.0,1942.0,517.0,0.0,0.0,0.0,89,8.0,65.0,16.0,...,0.053535,0.004812,0.039098,0.009624,0.0,0.0,0.0,0.035489,0.001479,4.271186


In [156]:
aggregates_over_time.isnull().sum()

activity_Nonproduction                       0
activity_Input                               0
activity_Remove/Cut                          0
activity_Replace                             0
activity_Paste                               0
activity_Move                                0
is_new_burst_start                           0
is_new_burst_start_Nonproduction             0
is_new_burst_start_Input                     0
is_new_burst_start_Remove/Cut                0
is_new_burst_start_Replace                   0
is_new_burst_start_Paste                     0
is_new_burst_start_Move                      0
is_thought_delimiting_punctuation            0
delete_insert_ratio                          0
word_count_delta_event                       0
preceding_pause_time                         0
latency_time_p50                             0
pause_time_p50                               0
has_cursor_position_moved_right_mean         0
word_count_delta_burst_mean                  0
word_count_de

In [157]:
events_by_window = (
    X_train
    .groupby(['id', 'window_30s'])
    [event_vars_sum]
    .agg(sum)
    .astype(float)
    .fillna(0)
    .reset_index(drop=False)
    )
events_by_window['window_30s_idx'] = events_by_window.index

events_by_window['delete_insert_ratio'] = (
    events_by_window['activity_Remove/Cut'] / 
    events_by_window['activity_Input'] 
    )


# for variability measure more comparable between writers, de-mean by writer. 
# Ex: higher-throughput writer incurs higher stddev, because values have higher magnitude
# join method allows for merge on one index column, of multiple possible
events_by_window = events_by_window.join(
    aggregates_over_time[[x + '_per_s' for x in event_vars_sum]],
    on='id',
    how='left'
    )
for var in event_vars_sum:
    events_by_window[var + '_time_norm'] = (
        events_by_window[var] / 
        (events_by_window[var + '_per_s'].replace(0, None) * 30)
        ).fillna(1)
events_by_window = events_by_window.drop(columns=[x + '_per_s' for x in event_vars_sum])

events_over_time_ren = aggregates_over_time[event_vars_sum]
events_over_time_ren.columns = [x + "_total" for x in events_over_time_ren.columns]
events_by_window = events_by_window.join(events_over_time_ren, on='id', how='left')
for var in event_vars_sum:
    events_by_window[var + '_frac_total'] = (
        events_by_window[var] / (events_by_window[var + '_total'].replace(0, None))
        ).fillna(1)
events_by_window = events_by_window.drop(columns=[x + '_total' for x in event_vars_sum])

In [158]:
conti_by_window = (
    X_train
    .assign()
    .groupby(['id', 'window_30s'])
    [conti_vars_sum]
    .agg(sum)
    .astype(float)
    .fillna(0)
    .reset_index(drop=False)
    )
conti_by_window['window_30s_idx'] = conti_by_window.index

conti_over_time_ren = aggregates_over_time[conti_vars_sum]
conti_over_time_ren.columns = [x + "_total" for x in conti_over_time_ren.columns]
conti_by_window = conti_by_window.join(conti_over_time_ren, on='id', how='left')
for var in conti_vars_sum:
    conti_by_window[var + '_frac_total'] = (
        conti_by_window[var] / conti_by_window[var + '_total']
        )
conti_by_window = conti_by_window.drop(columns=[x + '_total' for x in conti_vars_sum])

In [159]:
centrals_by_window = (
    X_train
    .assign()
    .groupby(['id', 'window_30s'])
    ['cursor_position_vs_max']
    .agg('mean')
    .astype(float)
    .reset_index(drop=False)
    )
centrals_by_window['window_30s_idx'] = centrals_by_window.index

In [160]:
aggregates_by_window = pd.merge(
    events_by_window, 
    conti_by_window,
    how='left'
    )

aggregates_by_window = pd.merge(
    aggregates_by_window, 
    centrals_by_window,
    how='left'
    )

In [161]:
from scipy.stats import entropy

entropy_vars = [var for var in aggregates_by_window.columns if 'frac_total' in var]
entropy_by_window = (
    aggregates_by_window
    .groupby(['id'])
    [entropy_vars]
    .agg(lambda x: entropy(x.value_counts()))
    )
entropy_by_window.columns = [
    x + '_entropy' 
    for x in entropy_by_window.columns
    ]


sd_by_window = (
    aggregates_by_window
    .drop(columns=['window_30s', 'window_30s_idx'])
    .groupby(['id'])
    .agg(np.std)
    )
sd_by_window.columns = [
    x + "_stddev"
    for x in sd_by_window.columns
    ]


trend_by_window = (
    aggregates_by_window
    .sort_values(['id', 'window_30s_idx'])
    .drop(columns=['window_30s'])
    .groupby(['id'])
    .corr()
    )
# extract correlations strictly with time index
trend_by_window = trend_by_window.xs('window_30s_idx', level=1)

vars_drop = (
    [x for x in trend_by_window.columns if 'time_norm' in x]
    + [x for x in trend_by_window.columns if 'frac_total' in x]
    + ['window_30s_idx']
    )
trend_by_window = trend_by_window.drop(columns=vars_drop)

trend_by_window.columns = [
    x + "_ttrend"
    for x in trend_by_window.columns
    ]

trend_by_window = trend_by_window.fillna(0)

In [162]:
vari_by_window = pd.merge(
    entropy_by_window,
    sd_by_window,
    how='left',
    left_index=True,
    right_index=True
    )   

vari_by_window = pd.merge(
    vari_by_window,
    trend_by_window,
    how='left',
    left_index=True,
    right_index=True
    )   

In [163]:
X_train_transform = pd.merge(
    aggregates_over_time,
    vari_by_window,
    how='left',
    left_index=True,
    right_index=True
    )

In [164]:
X_train_transform

,activity_Nonproduction,activity_Input,activity_Remove/Cut,activity_Replace,activity_Paste,activity_Move,is_new_burst_start,is_new_burst_start_Nonproduction,is_new_burst_start_Input,is_new_burst_start_Remove/Cut,...,is_new_burst_start_Input_ttrend,is_new_burst_start_Remove/Cut_ttrend,is_new_burst_start_Replace_ttrend,is_new_burst_start_Paste_ttrend,is_new_burst_start_Move_ttrend,is_thought_delimiting_punctuation_ttrend,delete_insert_ratio_ttrend,word_count_delta_event_ttrend,preceding_pause_time_ttrend,cursor_position_vs_max_ttrend
id,,,,,,,,,,,,,,,,,,,,,
001519c8,120.0,2010.0,417.0,7.0,0.0,3.0,125,23.0,89.0,13.0,...,-0.080663,-0.105717,0.0,0.0,0.0,-0.060836,0.264933,-0.111471,0.041532,-0.568160
0022f953,254.0,1938.0,260.0,1.0,1.0,0.0,81,32.0,41.0,8.0,...,-0.458482,0.118906,0.0,0.0,0.0,-0.443038,0.363823,-0.684394,0.252740,-0.534349
0042269b,175.0,3515.0,439.0,7.0,0.0,0.0,78,30.0,46.0,2.0,...,-0.271405,-0.155478,0.0,0.0,0.0,0.006654,0.031508,-0.037286,0.055098,-0.412200
0059420b,99.0,1304.0,151.0,1.0,1.0,0.0,88,9.0,77.0,2.0,...,-0.198990,0.112587,0.0,0.0,0.0,-0.080463,0.243067,-0.225482,-0.344996,-0.505044
0075873a,72.0,1942.0,517.0,0.0,0.0,0.0,89,8.0,65.0,16.0,...,-0.288949,-0.018794,0.0,0.0,0.0,-0.278883,-0.063513,-0.222532,-0.016912,-0.368961
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ffb8c745,189.0,3588.0,960.0,2.0,0.0,0.0,42,6.0,26.0,10.0,...,-0.398026,0.537128,0.0,0.0,0.0,-0.167464,0.331158,-0.328589,0.208079,-0.821336
ffbef7e5,148.0,2395.0,60.0,1.0,0.0,0.0,91,25.0,65.0,1.0,...,-0.319199,0.154109,0.0,0.0,0.0,-0.217964,0.183728,-0.173867,0.080927,-0.601130
ffccd6fd,126.0,2849.0,88.0,0.0,0.0,0.0,88,14.0,71.0,3.0,...,-0.058617,-0.011039,0.0,0.0,0.0,-0.046957,-0.085331,-0.302629,-0.144474,0.154743


In [165]:
X_train_transform.isnull().mean().to_dict()

{'activity_Nonproduction': 0.0,
 'activity_Input': 0.0,
 'activity_Remove/Cut': 0.0,
 'activity_Replace': 0.0,
 'activity_Paste': 0.0,
 'activity_Move': 0.0,
 'is_new_burst_start': 0.0,
 'is_new_burst_start_Nonproduction': 0.0,
 'is_new_burst_start_Input': 0.0,
 'is_new_burst_start_Remove/Cut': 0.0,
 'is_new_burst_start_Replace': 0.0,
 'is_new_burst_start_Paste': 0.0,
 'is_new_burst_start_Move': 0.0,
 'is_thought_delimiting_punctuation': 0.0,
 'delete_insert_ratio': 0.0,
 'word_count_delta_event': 0.0,
 'preceding_pause_time': 0.0,
 'latency_time_p50': 0.0,
 'pause_time_p50': 0.0,
 'has_cursor_position_moved_right_mean': 0.0,
 'word_count_delta_burst_mean': 0.0,
 'word_count_delta_burst_p50': 0.0,
 'cursor_position_vs_max_avg': 0.0,
 'length_latest_string_mean': 0.0,
 'length_latest_string_stddev': 0.0004046944556859571,
 'pause_time_max': 0.0,
 'initial_pause_time_max': 0.014973694860380412,
 'pause_time_p99': 0.0,
 'word_count_delta_burst_max': 0.0,
 'total_time': 0.0,
 'length_lates

In [166]:
CURSOR_POSITION_VS_MAX_STDDEV_P50 = 246.6

X_train_transform['cursor_position_vs_max_stddev'] = (
    X_train_transform['cursor_position_vs_max_stddev'].fillna(CURSOR_POSITION_VS_MAX_STDDEV_P50)
)

In [167]:
(
    X_train_transform
    .drop(columns='delete_insert_ratio_stddev')
    .to_pickle("./data/processed/X_train.pkl")
)